In [132]:
# as an idiot, i may have inadvertently not extracted columns, so i have ran the processing script again (v5), 
# removing the aspect ratio threshold + min diameter threshold to see if it has made a difference (i really hope not)

In [214]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
#import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
#import netCDF4 as nc
from datetime import datetime, timedelta
import shutil
import os
import zipfile

In [215]:
## function to reduce dataframe
def reduce_full_df(big_df, thresh_diam, include_aspect = True):
    if include_aspect != True:
        smaller_df = big_df[
        (big_df['Euclidean_d_max'] >= thresh_diam) &
        (big_df['first_diode_trunc'] == 0) &
        (big_df['last_diode_trunc'] == 0) &
        (big_df['image_trunc'] == 0)]

    else:
        smaller_df = big_df[
        (big_df['Euclidean_d_max'] >= thresh_diam) &
        (big_df['first_diode_trunc'] == 0) &
        (big_df['last_diode_trunc'] == 0) &
        (big_df['image_trunc'] == 0) &
        (big_df['aspect_ratio'] <= 10)]
    return smaller_df

### function to get matching files - v4 (in original place) and v5 (of script) in the v2 folder
# example file = '220731153602' 
# example probe = 'hvps' # hvps, 2ds/ch_0 , 2ds/ch_1
def matching_files(file_name, probe_name, reduced=True):
    v4_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/'+probe_name+'/' ## no min diameter + no aspect ratio limit (hvps threshold also increased)
    v5_path = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/'+probe_name+'/v2/'

    #print(v4_path)
    v4_loc = f'{v4_path}flight_{file_name}.csv'
    v5_loc = f'{v5_path}flight_{file_name}.csv'
    print(v4_loc)
    print(v5_loc)
    v4_csv = pd.read_csv(v4_loc)
    v5_csv = pd.read_csv(v5_loc)

    if reduced == False:
        return v4_csv, v5_csv

    else:
        if probe_name == 'hvps':
            min_diam = 3000
        else:
            min_diam = 300
            
        v4_reduced = reduce_full_df(v4_csv, min_diam)
        v5_reduced = reduce_full_df(v5_csv, min_diam)
        return v4_reduced, v5_reduced
    

In [216]:
### need to find the odd images and have a look at them maybe?
def zip_directory(directory_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(directory_path, '..')))
                
def not_matching_images(v4_df,v5_df, date, probe, folder_name = None, move_images = False):
    v4_name = list(v4_df['name'])
    v5_name = list(v5_df['name'])
    
    new_df = np.array(v5_name) 
    # Array of items to remove 
    og_df = np.array(v4_name)
    not_matching = list(new_df[~np.isin(new_df, og_df)]) # ones that are in v5 but not v4
    if move_images != False and len(not_matching) >= 1:
        image_loc = f'/gws/nopw/j04/dcmex/users/ezriab/processed_images/{probe}/v2/{date}/'
        save_path = f'/home/users/esree/2DS_code/particle_extraction/image_check_v4_v5/{date}_{folder_name}'

        if not os.path.exists(save_path):
            os.makedirs(save_path)

        for image in not_matching:
            image_full_loc = f'{image_loc}{image}.png'
            shutil.copy(image_full_loc, save_path+'/')

        ## then make zip file too
        zip_directory(save_path,save_path+'.zip')
        
    return not_matching

def print_results(v4,v5, probe_name):
    if probe_name == 'hvps':
            min_diam = 3000
    else:
        min_diam = 300
    v4_proper_aspect = reduce_full_df(v4, min_diam)
    v5_proper_aspect = reduce_full_df(v5, min_diam)

    v4_no_aspect = reduce_full_df(v4, min_diam, include_aspect = False)
    v5_no_aspect = reduce_full_df(v5, min_diam, include_aspect = False)

    print('original aspect ratio adhered, <= 10')
    v4_asp = len(v4_proper_aspect['name'])
    v5_asp = len(v5_proper_aspect['name'])
    print(f'v4 {v4_asp}')
    print(f'v5 {v5_asp}')

    print('')
    print('no aspect ratio limits')
    v4_no_asp = len(v4_no_aspect['name'])
    v5_no_asp = len(v5_no_aspect['name'])
    print(f'v4 {v4_no_asp}')
    print(f'v5 {v5_no_asp}')
    return v4_proper_aspect, v5_proper_aspect, v4_no_aspect, v5_no_aspect

In [220]:
## testing a bunch 
date_of_interest = '220723153143' # just full date
probe_file = 'hvps' # 'hvps' # hvps, 2ds/ch_0 , 2ds/ch_1
v4_test, v5_test = matching_files(date_of_interest,probe_file,reduced=False)
v4_asp_df, v5_asp_df, v4_no_asp_df, v5_no_asp_df = print_results(v4_test, v5_test, probe_file)

/gws/nopw/j04/dcmex/users/ezriab/processed_stats/hvps/flight_220723153143.csv
/gws/nopw/j04/dcmex/users/ezriab/processed_stats/hvps/v2/flight_220723153143.csv
original aspect ratio adhered, <= 10
v4 287
v5 302

no aspect ratio limits
v4 287
v5 346


In [213]:
# ones with original aspect ratio:
test_not_matching = not_matching_images(v4_asp_df, v5_asp_df, date_of_interest, probe_file, folder_name = 'prop_asp', move_images= True) 

# ones with no aspect ratio:
test_not_matching = not_matching_images(v4_no_asp_df, v5_no_asp_df, date_of_interest, probe_file, folder_name = 'no_asp', move_images= True)

In [217]:
og_all_im = '/gws/nopw/j04/dcmex/users/ezriab/processed_images/hvps/220723153143'
where_i_want_saved = '/home/users/esree/2DS_code/particle_extraction/image_check_v4_v5/220723153143_all.zip'
zip_directory(og_all_im,where_i_want_saved)

OSError: [Errno 116] Stale file handle

In [218]:
## move original images, to see what good ones look like
def move_good_image(v4_df, date, probe):
    folder_save_name = f'good_{date}'
    image_loc = f'/gws/nopw/j04/dcmex/users/ezriab/processed_images/{probe}/{date}/'
    save_path = f'/home/users/esree/2DS_code/particle_extraction/image_check_v4_v5/{folder_save_name}'
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    
    v4_name = list(v4_df['name'])
    for image in v4_name:
        image_full_loc = f'{image_loc}{image}.png'
        shutil.copy(image_full_loc, save_path+'/')

    ## then make zip file too
    zip_directory(save_path,save_path+'.zip')


In [221]:
#date_of_interest = '220723153143' # just full date
#probe_file = 'hvps' # 'hvps' # hvps, 2ds/ch_0 , 2ds/ch_1
move_good_image(v4_asp_df, date_of_interest, probe_file)